In [2]:
import Foundation
%include "engine.swift" 
%include "utils.swift"

// Type aliases are super useful and save a lot of time
// Especially when we are manually typing out tensors for testing purposes...
// typealias F = Float
// typealias T = Tensor

class Neuron {
    var w: [Value] 
    var b: Value
    var nonlin: Bool
    
    init(nin: Int, nonlin: Bool=true) {
        self.w = []
        for i in 1...nin {
            self.w.append(Value(data: (Float.random(in: -1..<1))))
        }
        self.b = Value(data: 0)
        self.nonlin = nonlin
    }
    
    func _call(_ inputs: [Value], _ show: Bool=false) -> Value {
        // (inputs • weights) + bias
        var wXi: [F] = []
        for (i, weight) in self.w.enumerated() {
            wXi.append(weight.data + inputs[i].data)
        }
        // sum(inputs • weights) + bias
        if (show) { print("(Inputs • Weights) + bias = \(sum(wXi))")} // optional validation
        var act = Value(data: sum(wXi) + self.b.data)
        if self.nonlin {
            return act._relu()
        } else {
            return act
        }
    }
    
    func _parameters(_ show: Bool=true) {
        for weight in self.w {
            weight.data += self.b.data
            if (show) {
                print(weight.data)
            }
        }
    }
}

In [3]:
var n1 = Neuron(nin: 3)
var inputs = [Value(data: 1.0), Value(data: 2.0), Value(data: -2.5), Value(data: -0.2)]
n1._parameters()
var result = n1._call(inputs, true)
print(result.data)

0.17892742
-0.6612606
-0.4214728
(Inputs • Weights) + bias = -0.40380597
0.0


In [4]:
class Layer {
    var neurons: [Neuron] = []
    
    init(nin: Int, nout: Int) {
        // Create nout number of neurons each with nin inputs
        for neuron in 1...nout {
            self.neurons.append(Neuron(nin: nin))
        }
    }
    
    func _call(_ inputs: [Value]) -> [Value] {
        var out: [Value] = []
        for neuron in self.neurons {
            out.append(neuron._call(inputs))
        }
        if out.count == 1 {
            var small_out: [Value] = []
            small_out.append(out[0])
            
            return small_out
        } 
        else {
            return out
        }
    }
    
    func _parameters(_show: Bool=true) {
        for neuron in self.neurons {
            print(type(of :neuron._parameters()))
        }
    }
    
    func _len() {
        print("\(self.neurons.count) Neurons")
    }
}

In [5]:
var inputs = [Value(data: 1.0), Value(data: 2.0), Value(data: -2.5), Value(data: -0.2)]
var l1 = Layer(nin: 4, nout: 12)

var output = l1._call(inputs)
for val in output {
    print(val.data)
}

1.6724458
0.0
1.1806254
0.0
0.0
0.0
1.1066272
3.503275
0.0
0.019615933
0.0
1.4238551


In [87]:
class MLP {
    var layers: [Layer] = []
    
    init() {
        layers.append(Layer(nin: 1, nout: 2)) // l1
//         layers.append(Layer(nin: 16, nout: 16)) // l2
//         layers.append(Layer(nin:16, nout: 1)) // final
    }
    
    func _call(_ inputs: [Value]) {
        for layer in self.layers {
            print(layer._parameters())
            layer._call(inputs)
            print(layer._parameters())
        }
    }
}

In [88]:
var inputs = [Value(data: 1.0), Value(data: 2.0)]
var mlp = MLP()
var x = [Value(data: 2.0)]
mlp._call(x)

0.98194623
()
0.78147316
()
()
0.98194623
()
0.78147316
()
()
